In [39]:
with open('day18_inputs.txt') as f:
    content = [l.strip('\n').split(' ') for l in f]

for l in content:
    print(l)

['2', '*', '3', '+', '(4', '*', '5)']
['2', '*', '3', '+', '(4', '*', '(1', '*', '2)', '+', '(2', '*', '3))']
['5', '+', '(8', '*', '3', '+', '9', '+', '3', '*', '4', '*', '3)']
['5', '*', '9', '*', '(7', '*', '3', '*', '3', '+', '9', '*', '3', '+', '(8', '+', '6', '*', '4))']


### Part 1 

Evaluate the expressions:
- Parentheses must be evaluated first
- BODMAS doesn't apply, precedence of calculations is just from left to right

e.g. with BODMAS, 5 + 3 * 2 would = 11, but in this case it would equal 16

#### First we get the nesting of the calculation

In [100]:
test = '5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))'.split(' ') #content[0]
test

['5',
 '*',
 '9',
 '*',
 '(7',
 '*',
 '3',
 '*',
 '3',
 '+',
 '9',
 '*',
 '3',
 '+',
 '(8',
 '+',
 '6',
 '*',
 '4))']

In [102]:
import copy

def get_nesting(calc_str):
    # Takes calc_str and returns a list of how many layers nested an item is in the string
    
    list_nesting = []
    nesting = 0
    for c in calc_str:
        # Nesting increases for a digit when it's inside open bracket(s)
        nesting += c.count('(')
        list_nesting.append(nesting)
        
        # Nesting decreases for the NEXT digit if the current digit has closed bracket(s)
        nesting -= c.count(')')
    
    return list_nesting


def operator_by_order(calc_list):
    # Given a calc_list, takes the first 3 elements and applies the middle operator
    # Returns an int
    # eg. ['3','+','5','*','6'] would be parsed as 3+5 and return 8
    a, op, b = calc_list[:3]
    return eval('{}{}{}'.format(a,op,b))


for i in range(3):
    
    # Assign calculation string
    calc_str = test
    
    # Get nesting at each position, then strip ( ) from the list
    calc_str_nesting = get_nesting(calc_str)
    calc_str_clean = [n.replace('(','').replace(')','') for n in calc_str]
    
    print('Original string:',' '.join(calc_str))
    print('Clean string:',calc_str_clean)
    print('Nesting:',calc_str_nesting)
    
    # Then, join each element of the calculation, it's nesting, and it's position
    calc_str_w_nesting_and_pos = list(zip(calc_str_clean, 
                                      calc_str_nesting, 
                                      [i for i in range(len(calc_str_clean))]))
    
    # Loop through each level of nesting, starting with max nesting
    for level in reversed(list(set(calc_str_nesting))):
        print('level',level)
        calc_str_filt = [c for c in calc_str_w_nesting_and_pos if c[1] == level]
        print(calc_str_filt)
        
        # Figure out how many separate equations to evaluate at this level of nesting
        calc_str_filt_w_tranches = []
        for i in range(len(calc_str_filt)):
            tranche = 0
            c = calc_str_filt[i]
            if i == 0:
                calc_str_filt_w_tranches.append([c[0], tranche])
            else:
                c_prior = calc_str_filt[i-1]
                if c[1] != c_prior[1] + 1:
                    tranche+=1
                calc_str_filt_w_tranches.append([c[0], tranche])
                    
        
        for i in range(tranche):
            calc_str_filt_tranche = [c[0] for c in calc_str_filt_w_tranches if c[1] == i]
            # Repeat until whole equation is evaluated
            while len(calc_str_filt_tranche) > 1:
                print('Calculating operation of first 3 elements, total length left: {}'.format(len(calc_str_filt_tranche)))
                calc_str_filt_new = [operator_by_order(calc_str_filt_tranche[:3])]
                # Append anything yet to be evaluated
                if len(calc_str_filt_tranche) > 3:
                    calc_str_filt_tranche_new.extend(calc_str_filt_tranche[3:])

                calc_str_filt_tranche = calc_str_filt_tranche_new
                print(calc_str_filt_tranche)

            calc_str_filt = str(calc_str_filt[0])
        
        
        # Create new calculation string using evaluated results
        calc_str_w_nesting_and_pos_new = []
        for i in range(len(calc_str_w_nesting_and_pos)):
            c = calc_str_w_nesting_and_pos[i]
            
            if i == 0 or c[1] < level:
                if c[1] == level:
                    calc_str_w_nesting_and_pos_new.append([calc_str_filt, level-1])
                if c[1] < level:
                    calc_str_w_nesting_and_pos_new.append([c[0], c[1]])
            else:
                c_prior = calc_str_w_nesting_and_pos[i-1]
                if c_prior[1] != level:
                    calc_str_w_nesting_and_pos_new.append([calc_str_filt, level-1])
            
            
        calc_str_w_nesting_and_pos = list(zip([c[0] for c in calc_str_w_nesting_and_pos_new],
                                              [c[1] for c in calc_str_w_nesting_and_pos_new],
                                              [i for i in range(len(calc_str_w_nesting_and_pos_new))]))
            
                    
        
        
    print(calc_str_w_nesting_and_pos)
    break

Original string: 5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))
Clean string: ['5', '*', '9', '*', '7', '*', '3', '*', '3', '+', '9', '*', '3', '+', '8', '+', '6', '*', '4']
Nesting: [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
level 2
['8', '+', '6', '*', '4']
Calculating operation of first 3 elements, total length left: 5
[14, '*', '4']
Calculating operation of first 3 elements, total length left: 3
[56]
level 1
['7', '*', '3', '*', '3', '+', '9', '*', '3', '+', '56']
Calculating operation of first 3 elements, total length left: 11
[21, '*', '3', '+', '9', '*', '3', '+', '56']
Calculating operation of first 3 elements, total length left: 9
[63, '+', '9', '*', '3', '+', '56']
Calculating operation of first 3 elements, total length left: 7
[72, '*', '3', '+', '56']
Calculating operation of first 3 elements, total length left: 5
[216, '+', '56']
Calculating operation of first 3 elements, total length left: 3
[272]
level 0
['5', '*', '9', '*', '272']
Calculating operation of firs